In [ ]:
import json
import requests
import os
import psycopg2

ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

def lambda_handler(event, context):
    #EXTRACT
    url_ZH = 'http://api.openweathermap.org/data/2.5/forecast?lat=47.3744489&lon=8.5410422&units=metric&appid=127619cee40c71033954846a9bfcf283'
    api_response = requests.get(url_ZH)
    Zurich = api_response.json()
    
    main_temps = []
    main_temp_min = []
    main_temp_max = []
    main_humidity = []
    weather_main = []
    weather_description = []
    clouds = []
    wind = []
    visibility = []
    precipitation = []
    rain = []
    snow = []
    datetime = []
    city = Zurich.get('city', {}).get('name')
    lon = Zurich.get('city', {}).get('coord', {}).get('lon')
    lat = Zurich.get('city', {}).get('coord', {}).get('lat')
    
    for i, entry in enumerate(Zurich['list']):
        if i % 8 == 0:  # select 1 value every 24 hours
            main_temps.append(entry['main'].get('temp'))
            main_temp_min.append(entry['main'].get('temp_min'))
            main_temp_max.append(entry['main'].get('temp_max'))
            main_humidity.append(entry['main'].get('humidity'))
            for weather_condition in entry['weather']:
                weather_main.append(weather_condition.get('main'))
                weather_description.append(weather_condition.get('description'))
            clouds.append(entry.get('clouds', {}).get('all'))
            wind.append(entry.get('wind', {}).get('speed'))
            visibility.append(entry.get('visibility'))
            precipitation.append(entry.get('pop'))
            rain.append(entry.get('rain', {}).get('3h'))
            snow.append(entry.get('snow', {}).get('3h'))
            datetime.append(entry.get('dt_txt'))
            
    #LOAD
    try:
        print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
        conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))

    except psycopg2.Error as e:
        print("Error: Could not make connection to the Postgres database")
        print(e)
    
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Error: Could not get curser to the Database")
        print(e)

    # Auto commit is very important
    conn.set_session(autocommit=True)
    
    cur.execute("CREATE TABLE IF NOT EXISTS weather_tab (main_temps REAL, main_temp_min REAL, main_temp_max REAL, main_humidity INTEGER, weather_main TEXT, weather_description TEXT, clouds INTEGER, wind REAL, visibility INTEGER, precipitation REAL, rain REAL, snow REAL, datetime TEXT, city TEXT, lon REAL, lat REAL);")
    
    try:
        for i in range(len(main_temps)):
            cur.execute("""
                INSERT INTO weather_tab (
                    main_temps, main_temp_min, main_temp_max, main_humidity, weather_main, weather_description,
                    clouds, wind, visibility, precipitation, rain, snow, datetime,
                    city, lon, lat
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """, (
                main_temps[i], main_temp_min[i], main_temp_max[i], main_humidity[i],
                weather_main[i], weather_description[i], clouds[i], wind[i],
                visibility[i], precipitation[i], rain[i], snow[i], datetime[i],
                city, lon, lat
            ))
    except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
    try:
        cur.execute("SELECT * FROM two;")
    except psycopg2.Error as e:
        print("Error: select *")
        print (e)
    
    row = cur.fetchone()
    while row:
        print(row)
        row = cur.fetchone()

    cur.close()
    conn.close()
    
    return main_temp_min, city
